# Investment Strategy

Date: Jan 25, 2022
Authors: af, jk, jv, gl

Code modified by MBAN BA2 Group 8

In [1]:
import pandas as pd
import numpy as np
import time
import statsmodels.api as sm
from tqdm import tqdm
import os
from scipy import stats

# set directory 
path_data = '/Users/youngjituen/Desktop/BAFI 508 - Data Driven Investments/Project Proposal/PortStrat/data/'

In [2]:
os.listdir(path_data)

['.DS_Store',
 'Compustat_Annual.csv',
 'CRSP_Monthly.csv',
 'F-F_Research_Data_Factors.CSV']

## Data Preparation & Momentum Calculation

In [3]:
print("Prepare CRSP file")
t = time.time() # record the current time, so we can measure how long the code takes to run

# load data
crsp = pd.read_csv(path_data+'CRSP_Monthly.csv')

# Have a look at the data
print(crsp.head())
print(crsp.dtypes)


### formatting ###
# make all variable names lowercase
crsp.columns = map(str.lower,crsp.columns)

# You should see that one of the important variables 'RET' (return) is not a number but 'object'.
# It is preferable to have this variable as a number, which Python denotes as float64 (float64 is just a special way of saying that a variable is a number)
# If you are interested search for 'floating point number'on internet. But it is computer-science issue!

# Changes the returns to number format. Non-numeric data will be NAN
crsp['ret'] = pd.to_numeric(crsp['ret'],errors='coerce') 

# Change the dateformat
crsp['date'] = pd.to_datetime(crsp['date'], format='%Y-%m-%d')

# Create separate 'year' and 'month' variables
crsp['year'] = crsp['date'].apply(lambda date: date.year)
crsp['month'] = crsp['date'].apply(lambda date: date.month)

# Calculate market cap
crsp['mktcap'] = crsp['shrout'] * crsp['prc'].abs()

### Calculate 6-month cumulative return of stock we will use for sorting ###
# Sort the DataFrame by 'permno', 'year', and 'month' to ensure proper order
crsp.sort_values(by=['permno', 'year', 'month'], inplace=True)

# Calculate the momentum using a rolling window of six months
crsp['momentum'] = crsp.groupby('permno')['ret'].shift(2).rolling(window=6).apply(lambda x: (1 + x).prod() - 1, raw=True)

### Some basic data cleaning ###
# keep only common shares
crsp = crsp[crsp['shrcd'].isin([10,11])]

# keep only stocks from NYSE, AMEX and NASDAQ
crsp = crsp[crsp['exchcd'].isin([1,2,3])]

# make sure that there are no duplicates
# usually, we would investigate why there are duplicates and then decide which observation we want to keep
#    For here, it is enough to simply drop the duplicates.
crsp = crsp.drop_duplicates(subset=['date','permno'])

print('Completed in %.1fs' % (time.time()-t)) # show how long it took to run this code block

Prepare CRSP file


/var/folders/1d/v9plw3yn5d76qsgycn4621240000gn/T/ipykernel_5130/3514735524.py:5: DtypeWarning: Columns (5,6,9,18,19,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp = pd.read_csv(path_data+'CRSP_Monthly.csv')


   PERMNO        date    NAMEENDT  SHRCD  EXCHCD   SICCD    NCUSIP TICKER  \
0   10000  1985-12-31         NaN    NaN     NaN     NaN       NaN    NaN   
1   10000  1986-01-31  1986-12-03   10.0     3.0  3990.0  68391610  OMFGA   
2   10000  1986-02-28         NaN   10.0     3.0  3990.0  68391610  OMFGA   
3   10000  1986-03-31         NaN   10.0     3.0  3990.0  68391610  OMFGA   
4   10000  1986-04-30         NaN   10.0     3.0  3990.0  68391610  OMFGA   

                      COMNAM SHRCLS  ... CFACSHR  ALTPRC SPREAD    ALTPRCDT  \
0                        NaN    NaN  ...     NaN -2.5625    NaN  1986-01-07   
1  OPTIMUM MANUFACTURING INC      A  ...     1.0 -4.3750  0.250  1986-01-31   
2  OPTIMUM MANUFACTURING INC      A  ...     1.0 -3.2500  0.250  1986-02-28   
3  OPTIMUM MANUFACTURING INC      A  ...     1.0 -4.4375  0.125  1986-03-31   
4  OPTIMUM MANUFACTURING INC      A  ...     1.0 -4.0000  0.250  1986-04-30   

        RETX    vwretd    vwretx    ewretd    ewretx    sprtrn

In [4]:
# Check that the momentum calculation was correct

# Filter crsp to only include permno x
x = 83852
crsp_one_stock = crsp[crsp['permno'] == x]
crsp_one_stock

# Display selected columns
crsp_one_stock[['permno', 'year', 'month', 'ret', 'momentum']]

,permno,year,month,ret,momentum
2999209,83852,1996,8,NaN,NaN
2999210,83852,1996,9,0.101124,NaN
2999211,83852,1996,10,0.387755,NaN
2999212,83852,1996,11,-0.161765,NaN
2999213,83852,1996,12,-0.140351,NaN
2999214,83852,1997,1,0.122449,NaN
2999215,83852,1997,2,-0.327273,NaN
2999216,83852,1997,3,-0.135135,NaN
2999217,83852,1997,4,-0.660156,-0.168540
2999218,83852,1997,5,1.298851,-0.346939


## Sample Selection

In [5]:
# Exclusions based on Jegadeesh and Titman (2001)

# group by year and month
crsp_grouped = crsp.groupby(['year', 'month'])

# calculate deciles of market capitalization for each year and month group
crsp['mktcap_decile'] = crsp_grouped['mktcap'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop'))

# filter out obervations belonging to the smallest decile for mktcap 
crsp_filtered = crsp[crsp['mktcap_decile'] != 0]

# remove stocks from crsp that are priced below $5
crsp_final = crsp_filtered[crsp_filtered['prc'] >=5]

In [6]:
# Check the number of observations in each year and month group
counts = crsp_final.groupby(['year', 'month']).size()
counts

year  month
1979  1        1883
      2        1850
      3        1881
      4        1909
      5        1876
               ... 
2023  8        2821
      9        2761
      10       2692
      11       2733
      12       2783
Length: 540, dtype: int64

## Subset Data into Time Periods 

In [7]:
# Subset 1: 'year' = 1986 until 'year' = 2006
crsp_1 = crsp_final[(crsp_final['year'] >= 1986) & (crsp_final['year'] <= 2006)]

# Subset 2: 'year' = 2007 until 'year' = 2015 
crsp_2 = crsp_final[(crsp_final['year'] >= 2007) & (crsp_final['year'] <= 2015)]

# Subset 3: 'year' = 2016 until 'year' = 2023 
crsp_3 = crsp_final[(crsp_final['year'] >= 2016) & (crsp_final['year'] <= 2023)]

## Portfolio Creation

In [8]:
# Define a function to create portfolios 
def portfolio_sort(df, sort_var, var, n):
    """
    This function is used to calculate portfolio returns.
    df: dataframe
    sort_var: the sorting variable
    var: the variable we are interested in
    n: number of portfolios 
    """
    df = df.dropna(subset = [sort_var]).copy()

    # In each year-month, we calculate ranks of sort variable and add them as a new column
    df["rank"] = df.groupby(['year', 'month'])[sort_var].transform(lambda x: pd.qcut(x, n, labels=False, duplicates='drop'))

    # we drop sample with missing value of interests
    df = df.dropna(subset = [var])

    # define the way we calculate average return
    def cal_avg_return(x, v):
        # Calculate the average return over the next 6 months, including the current month
        window = 6
        return np.mean(x[v].iloc[:window])

    # In each month, we calculate average monthly return
    ptf = df.groupby(["year", "month", "rank"]).apply(lambda x: cal_avg_return(x, var)).reset_index()

    ptf = ptf.sort_values(["year", "month", "rank"])
    ptf.columns = ["year", "month", "rank", "ret"]

    return ptf

In [9]:
# Create portfolios for 1986 to 2006
pft_1 = portfolio_sort(crsp_1, 'momentum', 'ret', 10)
pft_1 = pft_1.pivot_table(index=['year', 'month'], columns='rank', values='ret')

In [10]:
# Create portfolios for 2007 to 2015 
pft_2 = portfolio_sort(crsp_2, 'momentum', 'ret', 10)
pft_2 = pft_2.pivot_table(index=['year', 'month'], columns='rank', values='ret')

In [11]:
# Create portfolios for 2016 to 2023 
pft_3 = portfolio_sort(crsp_3, 'momentum', 'ret', 10)
pft_3 = pft_3.pivot_table(index=['year', 'month'], columns='rank', values='ret')

## Evaluation

In [12]:
# load Fama French monthly factors
ff = pd.read_csv(path_data+'F-F_Research_Data_Factors.csv')

# rename columns
ff.rename({'Mkt-RF':'ExMkt',
           'DATE':'date'},axis=1,inplace=True)

# date variables
ff['year'] = ff['date'] // 100
ff['month'] = ff['date'] % 100
ff.set_index('date',inplace=True)


### formatting ###
# FF data is in percent. Convert to simple returns
ff[['ExMkt', 'SMB', 'HML', 'RF']] /= 100

In [13]:
# merge portfolio returns for 1986 to 2006 with Fama French data 
ff_1 = pd.merge(pft_1,ff,on=['year','month'])

In [14]:
# merge portfolio returns for 2007 to 2015 with Fama French data 
ff_2 = pd.merge(pft_2,ff,on=['year','month'])

In [15]:
# merge portfolio returns for 2016 to 2023 with Fama French data 
ff_3 = pd.merge(pft_3,ff,on=['year','month'])

### CAPM and FF Regressions

In [16]:
# List of ff dataframes
ff_dfs = [ff_1, ff_2, ff_3]

# Loop through each ff dataframe
for i, ff_df in enumerate(ff_dfs, start=1):
    
    # Calculate winners minus losers 
    ff_df[10] = ff_df[9] - ff_df[1]
    
    # Calculate the excess returns
    nportfolios = 11
    
    for p in range(nportfolios):
        ff_df['ExRet_'+str(p)] = ff_df[p] - ff_df['RF']
    
    # Market model regressions
    table_capm = []
    for p in range(nportfolios):
        # Regress portfolio excess return on market excess return
        results = sm.OLS(ff_df['ExRet_'+str(p)],
                         sm.add_constant(ff_df['ExMkt'])).fit()
        
        # Collect results
        table_row = pd.DataFrame({'alpha': results.params['const'],
                                  'beta_mkt': results.params['ExMkt'],
                                  'alpha_t': results.tvalues['const'],
                                  'beta_t': results.tvalues['ExMkt'],
                                  'rmse': np.sqrt(results.mse_resid),
                                  'R2': results.rsquared},
                                 index=[p])
        
        table_capm.append(table_row)
    
    # Combine the results for all portfolios
    table_capm = pd.concat(table_capm, axis=0)
    table_capm.index.name = 'decile'
    
    # Save CAPM results to CSV
    capm_file_name = "CAPM_results_" + str(i) + ".csv"
    table_capm.to_csv(capm_file_name)
    
    # Show results for CAPM
    print("CAPM for ff_" + str(i) + "\n", table_capm)
    
    # Three Factor model regressions
    table_ff = []
    for p in range(nportfolios):
        # Regress portfolio excess return on market excess return
        results = sm.OLS(ff_df['ExRet_'+str(p)],
                         sm.add_constant(ff_df[['ExMkt','SMB','HML']])).fit()
        
        # Collect results
        table_row = pd.DataFrame({'alpha': results.params['const'],
                                  'beta_mkt': results.params['ExMkt'],
                                  'beta_size': results.params['SMB'],
                                  'beta_hml': results.params['HML'],
                                  'alpha_t': results.tvalues['const'],
                                  'beta_mkt_t': results.tvalues['ExMkt'],
                                  'beta_size_t': results.tvalues['SMB'],
                                  'beta_hml_t': results.tvalues['HML'],
                                  'rmse': np.sqrt(results.mse_resid),
                                  'R2': results.rsquared},
                                 index=[p])
        
        table_ff.append(table_row)
    
    # Combine the results for all portfolios
    table_ff = pd.concat(table_ff, axis=0)
    table_ff.index.name = 'decile'
    
    # Save Fama-French 3 results to CSV
    ff_file_name = "FF_results_" + str(i) + ".csv"
    table_ff.to_csv(ff_file_name)
    
    # Show results for Fama-French 3
    print("Fama-French 3 for ff_" + str(i) + "\n", table_ff)


CAPM for ff_1
            alpha  beta_mkt   alpha_t     beta_t      rmse        R2
decile                                                             
0       0.018034  1.370338  3.334088  11.288730  0.084924  0.337635
1       0.004888  1.201653  1.187893  13.013885  0.064598  0.403855
2       0.009874  0.942544  2.794773  11.887149  0.055472  0.361111
3       0.010627  0.869192  3.185484  11.609245  0.052379  0.350269
4       0.010429  0.852374  3.250593  11.838907  0.050369  0.359237
5       0.006891  0.749196  2.196360  10.640506  0.049259  0.311713
6       0.008780  0.806531  2.060838   8.435366  0.066891  0.221561
7       0.012584  1.016276  3.461518  12.456725  0.057076  0.382975
8       0.014307  0.979817  3.645656  11.125173  0.061615  0.331139
9       0.024268  1.297882  4.262334  10.157787  0.089389  0.292147
10      0.015618  0.096701  2.180808   0.601672  0.112439  0.001446
Fama-French 3 for ff_1
            alpha  beta_mkt  beta_size  beta_hml   alpha_t  beta_mkt_t  \
deci

### Further Analysis

In [17]:
# List of ff dataframes
ff_dfs = [ff_1, ff_2, ff_3]

# Loop through each ff dataframe
for i, ff_df in enumerate(ff_dfs, start=1):
    # Subset the excess return data 
    portfolios_exret = ff_df[['ExRet_0','ExRet_1','ExRet_2','ExRet_3', 'ExRet_4',
                              'ExRet_5','ExRet_6','ExRet_7','ExRet_8', 'ExRet_9']].copy()  # Use .copy() to create a copy of the DataFrame

    # Calculate winners minus losers 
    portfolios_exret['wml'] = portfolios_exret['ExRet_9'] - portfolios_exret['ExRet_0']

    # Calculate mean, std, skewness, kurtosis, and autocorrelation for each portfolio
    portfolio_stats = {}
    for column in portfolios_exret.columns:
        mean = portfolios_exret[column].mean()
        std = portfolios_exret[column].std()
        skew = portfolios_exret[column].skew()
        kurt = portfolios_exret[column].kurtosis()
        autocorr = portfolios_exret[column].autocorr()

        # Calculate t-statistic for difference from zero
        t_stat, _ = stats.ttest_1samp(portfolios_exret[column], 0)

        portfolio_stats[column] = {
            'Mean': mean,
            'Std': std,
            'Skewness': skew,
            'Kurtosis': kurt,
            'Autocorrelation': autocorr,
            't_statistic': t_stat
        }

    # Convert the dictionary to a DataFrame
    portfolio_stats_df = pd.DataFrame.from_dict(portfolio_stats, orient='index')

    # Save the DataFrame to a CSV file
    csv_filename = f"portfolio_stats_ff_{i}.csv"
    portfolio_stats_df.to_csv(csv_filename)
    
    # Display the DataFrame
    print("Portfolio Stats for ff_" + str(i))
    print(portfolio_stats_df)


Portfolio Stats for ff_1
             Mean       Std  Skewness   Kurtosis  Autocorrelation  t_statistic
ExRet_0  0.027057  0.104139  0.841889   2.722548         0.087486     4.124500
ExRet_1  0.012800  0.083498 -0.000018   1.238933        -0.006793     2.433481
ExRet_2  0.016081  0.069262  0.244414   3.150583         0.157779     3.685624
ExRet_3  0.016351  0.064852  0.226502   1.799335         0.142776     4.002298
ExRet_4  0.016041  0.062799 -0.337849   2.581168         0.152489     4.054906
ExRet_5  0.011824  0.059256  0.265526   2.377563         0.172085     3.167645
ExRet_6  0.014091  0.075663  1.837481  16.134651         0.063923     2.956307
ExRet_7  0.019276  0.072517  1.304841  11.343371         0.054499     4.219596
ExRet_8  0.020759  0.075189  1.013631   5.975579        -0.020670     4.382808
ExRet_9  0.032813  0.106034  0.664799   3.494230         0.010368     4.912544
wml      0.005756  0.119508 -0.346490   3.602938        -0.044486     0.764590
Portfolio Stats for ff_2
  